# Plot prediction and ground truth

In [ ]:
import numpy as np
import pandas as pd
import respiration.utils as utils

signals_dir = utils.dir_path('outputs', 'signals')

predictions_file = utils.join_paths(signals_dir, 'raft_predictions.csv')
predictions = pd.read_csv(predictions_file)
predictions['signal_uv'] = predictions['signal_uv'].apply(eval).apply(np.array)
predictions['signal_uv_std'] = predictions['signal_uv_std'].apply(eval).apply(np.array)
predictions['signal_v'] = predictions['signal_v'].apply(eval).apply(np.array)
predictions['signal_v_std'] = predictions['signal_v_std'].apply(eval).apply(np.array)
predictions.head()

# predictions_file = utils.join_paths(signals_dir, 'fine_tuned_predictions.csv')
# predictions = pd.read_csv(predictions_file)
# predictions['signal'] = predictions['signal'].apply(eval).apply(np.array)
# predictions.head()

In [ ]:
from respiration.dataset import VitalCamSet

dataset = VitalCamSet()

In [ ]:
subject = 'Proband23'
model = 'raft_large'
# model = '20240509_144815'

prediction = predictions[
    (predictions['model'] == model) &
    (predictions['subject'] == subject) &
    (predictions['roi'] == 'chest')]

signal_v = prediction['signal_v'].values[0]
sampling_rate = prediction['sampling_rate'].values[0]

gt_signal = dataset.get_breathing_signal(subject, '101_natural_lighting')
gt_signal = gt_signal[:len(signal_v)]

gt_signal.shape, signal_v.shape

In [ ]:
import matplotlib.pyplot as plt

# Plot the raw signals in two separate figures
fig, axs = plt.subplots(2, 1, figsize=(20, 10))

# Add some space between the two figures
plt.subplots_adjust(hspace=0.5)

axs[0].plot(gt_signal, label='Ground truth')
axs[0].set_title('Ground truth')
axs[0].legend()

axs[1].plot(signal_v, label='Prediction')
axs[1].set_title('Prediction')
axs[1].legend()

In [ ]:
import respiration.analysis as analysis

compare = analysis.SignalComparator(signal_v, gt_signal, sampling_rate, detrend_tarvainen=False)
compare.errors()

In [ ]:
plt.figure(figsize=(20, 5))
plt.plot(compare.ground_truth, label='Ground truth')
plt.plot(compare.prediction, label='Prediction')
plt.legend()
plt.show()

In [ ]:
from dtaidistance import dtw
from dtaidistance import dtw_visualisation as dtwvis

random_signal = np.random.rand(len(compare.ground_truth))

compare2 = analysis.SignalComparator(
    signal_v,
    gt_signal,
    sampling_rate,
    detrend_tarvainen=False,
    normalize_signal=True,
    filter_signal=True,
)

s1 = compare2.ground_truth
s2 = compare2.prediction

d, paths = dtw.warping_paths(s1, s2, use_c=True)
best_path = dtw.best_path(paths)
dtwvis.plot_warpingpaths(s1, s2, paths, best_path)